## 1. Set Up

In [24]:
def get_flight_data(file_path): 
    print("Processing ... " + file_path)
    xtree = et.parse(file_path)
    xroot = xtree.getroot()
    
    #M633 Supplementary Header
    header_s_633 = xroot[1]
    flight_origin_date = header_s_633[0].get('flightOriginDate')
    flight_number = header_s_633[0][0][1][0].text 
    dep_airport_name = header_s_633[0][1].get('airportName')
    arr_airport_name = header_s_633[0][2].get('airportName')
    
    aircraft_reg = header_s_633[1].get('aircraftRegistration')
    aircraft_subtype = header_s_633[1][0].get('airlineSpecificSubType')
    aircraft_model = header_s_633[1][0][0].text 
    
    #M633 FlightInfo
    flight_info = xroot[2]
    aircraft_reg = header_s_633[1].get('aircraftRegistration')
    call_sign = flight_info.get('aTCCallsign')
    
   # print("Aircraft Reg: ", aircraft_reg) 
   # print("Flight No: ", flight_number, " ATC Call Sign: ", call_sign, "Flight Origin Date: " , flight_origin_date)
   # print("Departure:", dep_airport_name, " --> Arrival:", arr_airport_name)
   # print("Aircraft Reg: " , aircraft_reg, "Aircraft Model: " , aircraft_model, " Aircraft SubType: " , aircraft_subtype) 
        
    #M633 FlightPlan Header
    flight_plan_header = xroot[4]
    performance_factor = flight_plan_header[1].text  
    fuel_flow_avg = flight_plan_header[2][0][0].text          # kg / hr
    fuel_flow_holding = flight_plan_header[2][1][0].text
    fms_route_name = flight_plan_header[3].get('fMSRouteName') #.text
    route_name = flight_plan_header[3].get('routeName')
    wind_direction_avg = flight_plan_header[3][0][0][0].text
    wind_speed_avg = flight_plan_header[3][0][1][0].text
    ground_distance = flight_plan_header[3][9][0].text
    air_distance = flight_plan_header[3][10][0].text
    greatcircle_distance = flight_plan_header[3][11][0].text
    
    #print("Route Name:" , fms_route_name, "(", route_name ,")" )
    #print("Wind Direction Avg (Deg):", wind_direction_avg, "Wind Direction Speed (Kt):", wind_speed_avg)
    #print("Distance (NM) - Ground: ", ground_distance, " Air: ", air_distance, " Great Circule: ", greatcircle_distance) 
    
    #M633 Weight Header
    weight_header = xroot[6]
    takeoff_weight = int(weight_header[3][0][0].text) #/2.205
    landing_weight = int(weight_header[4][0][0].text) #/2.205
    burn = int(takeoff_weight) - int(landing_weight)
    
    #print("Weight - Take Off (Kgs):", int(takeoff_weight), "Weight - Landing (Kgs):", int(landing_weight))
    #print("Fuel Burn (Kgs)", burn, "Performance Factor:" , performance_factor)
    #print("Fuel Flow Avg (Lb/h):" , fuel_flow_avg, "Fuel Flow Holding (Lb/h):" , fuel_flow_holding)
    
    # Waypoints
    waypoints = xroot[7]
    df_cols = ["sequence_no", "waypoint_name", 
               "latitude", "longitude", 
               "altitude", "elapsed_time", 
               #"mach_number", 
               "true_airspeed", 
               #"indicated_airspeed", 
               "engine_efficiency", "time", "burn_off", "c"]
    rows = []
    
    for wp in waypoints:
    #for wp in xroot.iter('Waypoint'): 
        sequence_no = int(wp.get('sequenceId'))
        waypoint_name = wp.get('waypointName')
        lat = float(wp[0].get('latitude'))/3600
        long = float(wp[0].get('longitude'))/3600
        altitude, burn_off, fuel_flow, mach_number, true_airspeed, indicated_airspeed = 0.0,0.0, 0.0, 0.0, 0.0, 0.0
        seconds = 0
        timestamp = '2023-05-23T04:36:00' # datetime.today()
        date_format = '%Y-%m-%dT%H:%M:%S'
        altitude = None
    
        for child in wp:
            if child.tag == '{http://aeec.aviation-ia.net/633}Altitude':
                altitude = (float(child[0][0].text)*100)/3.218  if child[0][0] is not None else 0
            if child.tag == '{http://aeec.aviation-ia.net/633}BurnOff':
                # burn off is in kg 
                burn_off = float(child[0][0].text) # /2.205
            if child.tag == '{http://aeec.aviation-ia.net/633}TimeOverWaypoint':
                time = child[0][0].text
                timestamp = datetime.strptime(time, date_format) # - timedelta(days=23)
                #cleaned_df['time'] = cleaned_df['time'] - timedelta(days=13)
            if child.tag == '{http://aeec.aviation-ia.net/633}TimeFromPreviousWaypoint':
                split = child[0][0].text.split("H")
                minutes = split[1] 
                seconds = float(minutes.split("M")[0])*60
                #else:
                #    fuel_flow = 0
                #time = time - timedelta(days=13)
            if child.tag == '{http://aeec.aviation-ia.net/633}MachNumber':
                mach_number = float(child[0][0].text) # if mach_no_node is not None else 0
            if child.tag == '{http://aeec.aviation-ia.net/633}TrueAirSpeed':
                true_airspeed = float(child[0][0].text)*0.514 # if mach_no_node is not None else 0
                # source data - KT
                # converted to True airspeed (m s-1)": "true_airspeed",
            if child.tag == '{http://aeec.aviation-ia.net/633}IndicatedAirSpeed':
                indicated_airspeed = float(child[0][0].text)*0.514 # if mach_no_node is not None else 0
                
        rows.append({"sequence_no": sequence_no,
                 "waypoint_name": waypoint_name,
                 "latitude": lat,
                 "longitude": long, 
                 "altitude": altitude,
                 "elapsed_time": seconds, 
                 "time": timestamp, 
                 "burn_off": burn_off,
              #   "mach_number": mach_number,
                  "true_airspeed": true_airspeed, 
              #   "indicated_airspeed": indicated_airspeed,
                 "engine_efficiency": 0.0
                })
    
    # Set Takeoff Weight 
    rows[0]['c'] = takeoff_weight
    rows[0]['altitude'] = 0
    rows[len(rows)-1]['altitude']=0
    
    out_df = pd.DataFrame(rows, columns = df_cols)
    new_col = 'c'
    
    def apply_func_decorator(func):
        prev_row = {}
        def wrapper(curr_row, **kwargs):
            val = func(curr_row, prev_row)
           # print(val)
           # print(prev_row)
            prev_row.update(curr_row)
           # print(prev_row)
            prev_row[new_col] = val
           # print(prev_row)
           # print(val)
            return val
        return wrapper
    
    @apply_func_decorator
    def aircraft_mass(curr_row, prev_row):
        #print(curr_row)
        #print(prev_row)
        return prev_row.get("c", takeoff_weight) - curr_row["burn_off"]
    
    #+ curr_row['b'] + prev_row.get('c', 0)
    out_df["aircraft_mass"] = out_df.apply(aircraft_mass, axis=1)
    
    # fuel flow 
    #"Fuel mass flow rate (kg s-1)": "fuel_flow",
    #Kilogram Per Second (kg/s) is a unit in the category of Mass flow rate. 
    #It is also known as kilogram/second, kilograms per second. 
    #This unit is commonly used in the SI unit system. Kilogram Per Second (kg/s) has a dimension of MT-1 where M is mass, and T is time.
    
    def fuel_flow(df):
        if df["burn_off"] > 0 and df["elapsed_time"] > 0:
            return df["burn_off"] / df["elapsed_time"]
        else:
            return 0
    
    out_df["fuel_flow"] = out_df.apply(fuel_flow, axis=1)
    
    # Engine Efficiency
    
    F=0.22  # Thrust 
    Q=43.24  #  mJ/kg.
    
    def engine_efficiency(df):
        V = int(df["indicated_airspeed"])*0.514
        # The knot (/nɒt/) is a unit of speed equal to one nautical mile per hour,
        # exactly 1.852 km/h (approximately 1.151 mph or 0.514 m/s).
        # 0.51444
        fuel_flow = df["fuel_flow"]  # kg / s
        if V > 0 and fuel_flow > 0:
            efficiency = (F*V)/(Q*fuel_flow)
            return efficiency
        else:
            return 0
    
    if engine_efficiency_input < 0: 
        out_df["engine_efficiency"] = out_df.apply(engine_efficiency, axis=1)
    else:
        out_df["engine_efficiency"] = engine_efficiency_input
    
    #out_df.info()
    out_df = out_df.drop('c', axis=1)
    cleaned_df = out_df.drop_duplicates(subset='time', keep="first")
    #cleaned_df.describe()
    # max_altitude = ?
    # max_speed = ? 

    # demo synthetic flight
    flight_attrs = {
        "flight_id": call_sign,
        # set constants along flight path
        #"true_airspeed": true_airspeed_input, 
        "thrust": thrust_setting_input, 
        "nvpm_ei_n": non_volatile_emissions_index_input
    }

    wingspan_input = 37.57  
    if aircraft_model in wingspan_dict:
        wingspan_input =  wingspan_dict[aircraft_model]
        #print(wingspan_input, aircraft_model)
        
    aircraft = Aircraft(aircraft_type=aircraft_model, wingspan=wingspan_input, n_engine=n_engine_input)
    input_df = cleaned_df.interpolate(method='linear', limit_direction='forward', axis=0)
    input_df = input_df.fillna(0)
    
    #if (interpolate):
    #    print("interpolating")
    #    input_df = cleaned_df.interpolate(method='linear', limit_direction='forward', axis=0)
    #    input_df = input_df.fillna(0)
    #else: 
    #    input_df = cleaned_df
    #print(input_df)
    
    flight = Flight(input_df, aircraft=aircraft, attrs=flight_attrs)
    return flight, call_sign, flight_number, flight_origin_date, dep_airport_name, arr_airport_name, greatcircle_distance, fms_route_name, burn, aircraft_model, performance_factor, aircraft_subtype


def get_met_data(timebound):
    era5pl = ERA5(
        time=timebound,
        variables=Cocip.met_variables + Cocip.optional_met_variables,
        pressure_levels=pressure_levels,
    )
    era5sl = ERA5(time=timebound, variables=Cocip.rad_variables)
        
    met = era5pl.open_metdataset()
    rad = era5sl.open_metdataset()
    return met, rad

def compute_cocip(flight, met, rad):
    params = {
        "process_emissions": False,
        "verbose_outputs": True,
        #"dt_integration": np.timedelta64(10, "m"),
        "humidity_scaling": ConstantHumidityScaling(rhi_adj=0.98), 
    }
    cocip = Cocip(met=met, rad=rad, params=params)
    fl_out = cocip.eval(source=flight)
    flight_statistics = cocip.output_flight_statistics()
    return fl_out, flight_statistics


## 2. Input

In [17]:
#default values
engine_efficiency_input = 0.45
non_volatile_emissions_index_input =  1500000000000000 #1.897462e15 # nvpm_ei_n
n_engine_input = 2
thrust_setting_input = 0.22 # 
pressure_levels = [200,300,250,400,500,600]
interpolate = True
wingspan_dict = {
  "A320": 37.57,
  "B737": 36.0,
  "A350": 65,
  "A380": 80
}

## 3. Run Calculation Process 

In [35]:
import xml.etree.ElementTree as et 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
    
from pycontrails import Flight, Aircraft
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.cocip import Cocip
from pycontrails.models.humidity_scaling import ConstantHumidityScaling
        
from os import listdir
from os.path import isfile, join
mypath = './data/AAY_Test'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

print(onlyfiles)

output_cols = ["file_name", "call_sign", "flight_number", "flight_origin_date", "departure_time", "dep_airport_name",
               "arr_airport_name", "greatcircle_distance", "flight_time_duration", "fms_route_name", "aircraft_model",  
               "aircraft_subtype",
               "fuel_burn", "fuel_burn_src", "max_altitude", "max_true_airspeed", "avg_fuel_flow",
               "num_waypoints", "performance_factor", "short_lived_contrails", "persistent_contrails",
               "initial_contrail_length_km", "persistent_contrail_length_km",
               "bc_ei_n", "total_contrail_EF", "rhi_life_mean", "tau_contrail_mean", "sdr_mean_w", "rsr_mean_w", "olr_mean_w"]

output_rows = []

for file in onlyfiles:
    try: 
        
        file_path = "./data/AAY_Test/" + file
        flight, call_sign, flight_number, flight_origin_date, dep_airport_name, arr_airport_name, greatcircle_distance, fms_route_name, fuel_burn_src, aircraft_model, performance_factor,aircraft_subtype = get_flight_data(file_path)
    
        f = flight.dataframe
        fuel_burn = f['burn_off'].sum()
        avg_fuel_flow = f['fuel_flow'].mean()
        max_true_airspeed = f['true_airspeed'].max()
        max_altitude = f['altitude'].max()
        num_waypoints = len(f)
        departure_time = f.loc[0]['time']
        arrival_time = f.loc[len(f)-1]['time']
        duration_seconds = arrival_time - departure_time
        duration_minutes = duration_seconds.total_seconds() / 60
    
        timebound = (
            pd.to_datetime(flight["time"][0]).floor("H"),
            pd.to_datetime(flight["time"][-1]).ceil("H"), #+ pd.Timedelta("10H"),
        )
        era5pl = ERA5(
                time=timebound,
                variables=Cocip.met_variables + Cocip.optional_met_variables,
                pressure_levels=pressure_levels,
            )
        era5sl = ERA5(time=timebound, variables=Cocip.rad_variables)
        met = era5pl.open_metdataset()
        rad = era5sl.open_metdataset()
    
        fl_out, flight_statistics = compute_cocip(flight, met, rad) # pass met and rad ? 
        #display_results(flight_statistics)
        #print("---------------------")
    
        bc_ei_n = flight_statistics['BC EI_n mean (kg-1)']
        total_contrail_EF = flight_statistics['Total contrail EF (J)']
        short_lived_contrails = flight_statistics['Short-lived contrails']
        initial_contrail_length_km = flight_statistics['Initial contrail length (km)']
        persistent_contrails = flight_statistics['Persistent contrails']
        persistent_contrail_length_km = flight_statistics['Persistent contrail length (km)']
        rhi_life_mean = flight_statistics['RHi lifetime, Mean']
        tau_contrail_mean = flight_statistics['Tau contrail, Mean']
        sdr_mean_w = flight_statistics['SDR mean (W m-2)']
        rsr_mean_w = flight_statistics['RSR mean (W m-2)']
        olr_mean_w = flight_statistics['OLR mean (W m-2)']
    
        output_rows.append({"flight_number": flight_number, 
                     "call_sign": call_sign,
                     "flight_origin_date": flight_origin_date,
                     "dep_airport_name": dep_airport_name,
                     "arr_airport_name": arr_airport_name,
                     "departure_time": departure_time,
                     "flight_time_duration": duration_minutes,
                     "greatcircle_distance": greatcircle_distance,
                     "fms_route_name": fms_route_name,
                     "fuel_burn": fuel_burn,
                     "fuel_burn_src": fuel_burn_src,
                     "aircraft_model": aircraft_model, 
                     "aircraft_subtype": aircraft_subtype,
                     "avg_fuel_flow": avg_fuel_flow,
                     "max_true_airspeed": max_true_airspeed,
                     "max_altitude":  max_altitude,
                     "num_waypoints": num_waypoints,
                     "performance_factor": performance_factor,
                     "bc_ei_n": bc_ei_n,
                     "total_contrail_EF": total_contrail_EF,
                     "short_lived_contrails": short_lived_contrails, 
                     "initial_contrail_length_km": initial_contrail_length_km,
                     "persistent_contrails": persistent_contrails, 
                     "persistent_contrail_length_km": persistent_contrail_length_km, 
                     "rhi_life_mean": rhi_life_mean,
                     "tau_contrail_mean": tau_contrail_mean, 
                     "file_name": file, 
                     "sdr_mean_w": sdr_mean_w,
                     "rsr_mean_w": rsr_mean_w,
                     "olr_mean_w": olr_mean_w,
                     })
    
    except Exception:
        print("Something went wrong") 
        continue 
        
    summary_df = pd.DataFrame(output_rows, columns = output_cols)
    
# has_contrails_how_many ?

['AAY-0053-090418z-KGPI-R01-arinc633.xml', 'AAY-0053-120418z-KGPI-R04-arinc633.xml', 'AAY-0166-101400z-KABE-R01-arinc633.xml', 'AAY-0191-092006z-KABE-R10-arinc633.xml', 'AAY-0191-111200z-KABE-R01-arinc633.xml', 'AAY-0191-122006z-KABE-R03-arinc633.xml', 'AAY-0194-091200z-KABE-R01-arinc633.xml', 'AAY-0194-121200z-KABE-R02-arinc633.xml', 'AAY-0362-091115z-KCVG-R01-arinc633.xml', 'AAY-0362-121115z-KCVG-R02-arinc633.xml', 'AAY-1142-121100z-KPIT-R03-arinc633.xml', 'AAY-1145-112136z-KPIT-R01-arinc633.xml', 'AAY-1401-091310z-KTYS-R03-arinc633.xml', 'AAY-1401-121310z-KTYS-R01-arinc633.xml', 'AAY-1785-101836z-KABE-R01-arinc633.xml', 'AAY-1810-111951z-KABE-R03-arinc633.xml', 'AAY-2748-091100z-KPIT-R01-arinc633.xml']
Processing ... ./data/AAY_Test/AAY-0053-090418z-KGPI-R01-arinc633.xml


C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\cocip.py:588: UserWarning: Flight trajectory has segment lengths close to or exceeding the 'max_seg_length_m' parameter. Evolved contrail segments may reach their end of life artificially early. Either resample the flight with the 'resample_and_fill' method (recommended), or use a larger 'max_seg_length_m' parameter. Current values: max_seg_length_m=40000.0, max_seg_length_on_trajectory=494430.96997134463
  warnings.warn(


Processing ... ./data/AAY_Test/AAY-0053-120418z-KGPI-R04-arinc633.xml


C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\cocip.py:588: UserWarning: Flight trajectory has segment lengths close to or exceeding the 'max_seg_length_m' parameter. Evolved contrail segments may reach their end of life artificially early. Either resample the flight with the 'resample_and_fill' method (recommended), or use a larger 'max_seg_length_m' parameter. Current values: max_seg_length_m=40000.0, max_seg_length_on_trajectory=494430.96997134463
  warnings.warn(
C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\wake_vortex.py:158: RuntimeWarning: divide by zero encountered in divide
  return (c * aircraft_mass * constants.g) / (wingspan**2 * rho_air * true_airspeed * n_bv)
C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\cocip.py:840: RuntimeWarning: divide by zero encountered in divide
  self._sac_flight.get_data_or_at

Processing ... ./data/AAY_Test/AAY-0166-101400z-KABE-R01-arinc633.xml


...

C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\cocip.py:588: UserWarning: Flight trajectory has segment lengths close to or exceeding the 'max_seg_length_m' parameter. Evolved contrail segments may reach their end of life artificially early. Either resample the flight with the 'resample_and_fill' method (recommended), or use a larger 'max_seg_length_m' parameter. Current values: max_seg_length_m=40000.0, max_seg_length_on_trajectory=163670.85251008163
  warnings.warn(
C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\output\flight_summary.py:75: RuntimeWarning: Mean of empty slice
  return np.nanmean(df_wypt_var[var_name])
C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\output\flight_summary.py:75: RuntimeWarning: Mean of empty slice
  return np.nanmean(df_wypt_var[var_name])
C:\Users\zaki.patel\AppData\Local\Programs\Python

Something went wrong
Processing ... ./data/AAY_Test/AAY-1810-111951z-KABE-R03-arinc633.xml


2023-06-20 18:53:58,650 INFO Welcome to the CDS
2023-06-20 18:53:58,652 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-06-20 18:53:58,854 INFO Request is queued
2023-06-20 18:54:00,083 INFO Request is running
2023-06-20 18:54:07,763 INFO Request is completed
2023-06-20 18:54:07,764 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data0/adaptor.mars.internal-1687301644.3047318-1348-6-933e60e1-560d-48f5-94da-ac58ad0e36ff.nc to C:\Users\ZAKI~1.PAT\AppData\Local\Temp\95ffb412090e670de5bb73ebeabb6b902b58f22764b148d9 (95.1M)
2023-06-20 18:54:25,662 INFO Download rate 5.3M/s                                                                      
2023-06-20 18:54:28,654 INFO Welcome to the CDS
2023-06-20 18:54:28,656 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-06-20 18:54:28,858 INFO Request is queued
2023-06-20 18:54:30,087 INFO 

Processing ... ./data/AAY_Test/AAY-2748-091100z-KPIT-R01-arinc633.xml


C:\Users\zaki.patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\pycontrails\models\cocip\cocip.py:588: UserWarning: Flight trajectory has segment lengths close to or exceeding the 'max_seg_length_m' parameter. Evolved contrail segments may reach their end of life artificially early. Either resample the flight with the 'resample_and_fill' method (recommended), or use a larger 'max_seg_length_m' parameter. Current values: max_seg_length_m=40000.0, max_seg_length_on_trajectory=253201.9537953088
  warnings.warn(


## 4. Review Summary

In [36]:
summary_df.describe()

,departure_time,flight_time_duration,fuel_burn,fuel_burn_src,max_altitude,max_true_airspeed,avg_fuel_flow,num_waypoints,initial_contrail_length_km,persistent_contrail_length_km,bc_ei_n,total_contrail_EF,rhi_life_mean,tau_contrail_mean,sdr_mean_w,rsr_mean_w,olr_mean_w
count,16,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,1.600000e+01,1.600000e+01,16.000000,16.000000,16.000000,16.000000,16.000000
mean,2023-06-11 02:59:18.750000128,112.437500,11274.125000,11326.375000,11167.650715,234.801625,1.467799,25.375000,561.708033,247.227666,1.500000e+15,-1.518575e+10,0.490310,0.134646,222.961020,100.667219,91.173837
min,2023-06-09 04:49:00,82.000000,8408.000000,8408.000000,10565.568676,231.814000,1.179583,20.000000,0.000000,0.000000,1.500000e+15,-3.512255e+12,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2023-06-09 13:03:30,108.000000,10729.250000,10795.000000,11109.384711,233.870000,1.369539,22.000000,389.898242,0.000000,1.500000e+15,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2023-06-11 16:09:00,120.500000,11778.000000,11800.500000,11187.072716,234.641000,1.466248,23.500000,620.999538,0.000000,1.500000e+15,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2023-06-12 11:18:15,124.000000,12507.500000,12533.500000,11187.072716,235.926000,1.541969,30.000000,892.990775,494.430970,1.500000e+15,0.000000e+00,1.101091,0.233211,369.234109,185.887311,189.395309
max,2023-06-12 20:17:00,126.000000,13259.000000,13259.000000,12119.328776,236.954000,1.878297,31.000000,1003.477562,901.755434,1.500000e+15,3.146033e+12,1.230235,0.488527,1031.297558,529.993167,245.123566
std,NaN,16.661207,1689.578009,1690.501617,367.139188,1.504054,0.174065,4.209117,335.497038,310.204010,0.000000e+00,1.217784e+12,0.576114,0.191756,374.822616,170.618894,108.557930


## 5 Download to CSV

In [37]:
filename = "allegiant_test_set_1"

In [38]:
import os, time
os.makedirs('./output', exist_ok=True)  
epoch_time = int(time.time())
summary_df.to_csv('./output/' + filename + str(epoch_time) + '.csv')